In [1]:
import requests #equivale ao httr
from bs4 import BeautifulSoup #equivale ao rvest
import json # Equivalente ao jsonlite
import time # usamos para pausar o script entre as buscas (evita bloqueio do Google).

In [22]:
import requests
from bs4 import BeautifulSoup

def buscar_noticias_google(termo_busca):
    termo_busca = termo_busca.replace(" ", "+")
    url = f"https://www.google.com/search?q={termo_busca}&tbm=nws&hl=pt-BR"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    resultados = []

    for item in soup.find_all("div", class_="dbsr"):
        try:
            titulo = item.find("div", class_="JheGif nDgy9d").get_text()
            link = item.a["href"]
            trecho = item.find("div", class_="Y3v8qd").get_text()
            resultados.append({"titulo": titulo, "link": link, "trecho": trecho})
        except AttributeError:
            # Pula se algum elemento não for encontrado
            continue

    return resultados

# Exemplo de uso #🔹 Aqui a gente extrai 3 coisas de cada bloco:
#titulo: o título da notícia.
#link: o endereço (URL) para clicar.
#trecho: um resuminho da matéria.
noticias = buscar_noticias_google("problemas na construção civil")
for i, noticia in enumerate(noticias, start=1):
    print(f"{i}. {noticia['titulo']}")
    print(f"   Link: {noticia['link']}")
    print(f"   Trecho: {noticia['trecho']}\n")
#Agora guardamos essas 3 informações em um dicionário ({}) e jogamos na lista resultados.     
    resultados.append({
            "titulo": titulo,
            "link": link,
            "trecho": trecho
        })


In [23]:
#Rodando a busca com várias palavras chaves 
buscas = [
    "problemas construção civil Itapema",
    "construção civil litoral SC",
    "desafios obras Santa Catarina"
]
# Aqui está a lista de frases que vamos buscar no Google Notícias.

In [24]:
#Lista para todas as noticias encontradas
todas_noticias = []

In [25]:
#Para cada frase da lista, a gente:
for termo in buscas:
    print(f"\n🔍 Buscando por: {termo}")#Mostra o que está buscando.
    noticias = buscar_noticias_google(termo) #Chama a função buscar_noticias_google().
    todas_noticias.extend(noticias) #Junta os resultados na lista todas_noticias.


🔍 Buscando por: problemas construção civil Itapema

🔍 Buscando por: construção civil litoral SC

🔍 Buscando por: desafios obras Santa Catarina


In [27]:
#Para cada notícia imprime 
for noticia in noticias:
        print(f"\n📰 {noticia['titulo']}\n{noticia['trecho']}\n🔗 {noticia['link']}")

In [29]:
#Espera 3 segundos para o google não te bloquear por excesso de requisição
time.sleep(3)  # Evita bloqueios

In [ ]:
#Resultado final 
#Você verá uma lista de notícias relevantes, prontas para serem lidas ou salvas.

#🔍 Buscando por: problemas construção civil Itapema

#📰 Crescimento desordenado gera problemas estruturais
#A falta de planejamento urbano em Itapema preocupa especialistas.
#🔗 https://noticia.com/link-exemplo


In [30]:
#Salvando em csv 
import csv

# Criar (ou sobrescrever) um arquivo CSV
with open("noticias_construcao.csv", mode="w", newline="", encoding="utf-8") as arquivo_csv:
    escritor = csv.writer(arquivo_csv)
    
    # Escreve o cabeçalho
    escritor.writerow(["Título", "Trecho", "Link"])

    # Escreve cada linha com os dados das notícias
    for noticia in todas_noticias:
        escritor.writerow([noticia["titulo"], noticia["trecho"], noticia["link"]])

print("\n✅ Arquivo 'noticias_construcao.csv' salvo com sucesso!")



✅ Arquivo 'noticias_construcao.csv' salvo com sucesso!


In [31]:
#Filtragem das noticias por palavra-chave
palavras_chave = ["irregularidade", "obra parada", "riscos", "colapso", "desabamento"]

noticias_filtradas = []

for noticia in todas_noticias:
    texto = (noticia["titulo"] + " " + noticia["trecho"]).lower()
    if any(palavra in texto for palavra in palavras_chave):
        noticias_filtradas.append(noticia)


In [32]:
#imprimindo as noticias por palavra-chave
print(f"\n🔎 {len(noticias_filtradas)} notícias relevantes encontradas:")

for noticia in noticias_filtradas:
    print(f"\n📰 {noticia['titulo']}\n{noticia['trecho']}\n🔗 {noticia['link']}")



🔎 0 notícias relevantes encontradas:


In [ ]:
#------- script por data mais recente

In [33]:
#Filtrando pelas notícias mais recentes
import requests
from bs4 import BeautifulSoup
import csv

# 1. Pesquisa no Google Notícias
query = "problemas construção civil Itapema OR litoral SC site:g1.globo.com OR site:ndmais.com.br OR site:diarinho.net"
url = f"https://www.google.com/search?q={query}&tbm=nws&hl=pt-BR"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# 2. Extrair resultados
noticias = soup.select("div.dbsr")
todas_noticias = []

for item in noticias:
    titulo = item.select_one("div.JheGif.nDgy9d").text
    trecho = item.select_one("div.Y3v8qd").text
    link = item.a["href"]

    # Aqui tentamos extrair a data relativa (ex: "há 2 dias")
    data_tag = item.find("span", class_="WG9SHc") # span.WG9SHc > span captura a data relativa.
    if data_tag and data_tag.span:
        data_relativa = data_tag.span.text
    else:
        data_relativa = "Data não encontrada"

    todas_noticias.append({
        "titulo": titulo,
        "trecho": trecho,
        "link": link,
        "data": data_relativa
    })

# 3. Salvar no CSV
with open("noticias_construcao.csv", mode="w", newline="", encoding="utf-8") as arquivo_csv:
    escritor = csv.writer(arquivo_csv)
    escritor.writerow(["Título", "Trecho", "Link", "Data"])
    for noticia in todas_noticias:
        escritor.writerow([noticia["titulo"], noticia["trecho"], noticia["link"], noticia["data"]])

print("\n✅ Arquivo 'noticias_construcao.csv' salvo com sucesso!")

# 4. Mostrar as notícias mais recentes (últimos 7 dias)
print("\n🔎 Notícias recentes (últimos 7 dias):")
#O filtro if any(x in noticia["data"] for x in ["hora", "dia", "minuto"]) exibe as notícias recentes, com base nessas palavras.
for noticia in todas_noticias:
    if any(x in noticia["data"] for x in ["hora", "dia", "minuto"]): 
        print(f"\n📰 {noticia['titulo']}\n📅 {noticia['data']}\n🔗 {noticia['link']}")



✅ Arquivo 'noticias_construcao.csv' salvo com sucesso!

🔎 Notícias recentes (últimos 7 dias):


In [34]:
print(response.status_code)    # Deve ser 200
print(response.text[:1000])    # Para ver se o HTML está como esperado


200
<!doctype html><html lang="pt-BR"><head><meta charset="UTF-8"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>problemas construção civil Itapema OR litoral SC site:g1.globo.com OR site:ndmais.com.br OR site:diarinho.net - Pesquisa Google</title><script nonce="U7KmFJFm1MsTn-D3eED1ng">(function(){
document.documentElement.addEventListener("submit",function(b){var a;if(a=b.target){var c=a.getAttribute("data-submitfalse");a=c==="1"||c==="q"&&!a.elements.q.value?!0:!1}else a=!1;a&&(b.preventDefault(),b.stopPropagation())},!0);document.documentElement.addEventListener("click",function(b){var a;a:{for(a=b.target;a&&a!==document.documentElement;a=a.parentElement)if(a.tagName==="A"){a=a.getAttribute("data-nohref")==="1";break a}a=!1}a&&b.preventDefault()},!0);}).call(this);(function(){window.google=window.google||{};var a=window.performance&&window.performance.timing&&"navigationStart"in window.performance.timing,b=google.stvsc&&google.st